In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Building our list of Wards and their constituencies

In [3]:
# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/Nairobi_County"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
canadatable=soup.find('table',{'class':"wikitable"})
df=pd.read_html(str(canadatable))
df=pd.DataFrame(df[0])
df.head()

,Constituency,Wards
0,Westlands,Kitisuru · Parklands/Highridge · Kangemi · Kar...
1,Dagoretti North,Kilimani · Kawangware · Gatina · Kileleshwa · ...
2,Dagoretti South,"Mutu-ini, Ngand'o · Riruta · Uthiru/Ruthimitu ..."
3,Langata,Karen · Nairobi West · Nyayo Highrise · South C
4,Kibra,Laini Saba · Lindi · Makina · Woodley-Kenyatta...


In [4]:
df2 = pd.DataFrame(df.Wards.str.split('·').tolist(), index=df.Constituency).stack()
df2 = df2.reset_index()[[0, 'Constituency']] # var1 variable is currently labeled 0
df2.columns = ['Ward','Constituency'] # renaming var1
df2

,Ward,Constituency
0,Kitisuru,Westlands
1,Parklands/Highridge,Westlands
2,Kangemi,Westlands
3,Karura,Westlands
4,Mountain View,Westlands
5,Kilimani,Dagoretti North
6,Kawangware,Dagoretti North
7,Gatina,Dagoretti North
8,Kileleshwa,Dagoretti North
9,Kabiro,Dagoretti North


### Acquiring the geographical co ordinates for our list

In [5]:
rawdata = [];
# loop to fetch latitude and longitude - if they are not found using the initial ward name, 2 more variations of the ward name are used to search.
for i in range(len(df2)):
    try:
        ward = df2.loc[i, "Ward"]
        constit = df2.loc[i, "Constituency"]
        address = '{}, {}, Nairobi, Kenya'.format(ward,constit)
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        rawdata.append([(ward,constit,latitude,longitude)])
    
    except:
        try:
            ward = ward.split('/')[0]
            address = '{}, {}, Nairobi'.format(ward,constit)
            geolocator = Nominatim(user_agent="ny_explorer")
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
            rawdata.append([(ward,constit,latitude,longitude)])
        
        except:
            try:
                address = '{}, Nairobi, Kenya'.format(ward)
                geolocator = Nominatim(user_agent="ny_explorer")
                location = geolocator.geocode(address)
                latitude = location.latitude
                longitude = location.longitude
                rawdata.append([(ward,constit,latitude,longitude)])
            
            except:
                pass
    
rawdata

 Parklands Westlands
 Mountain View Westlands
Kilimani  Dagoretti North
 Kawangware  Dagoretti North
 Gatina  Dagoretti North
 Kileleshwa  Dagoretti North
 Kabiro Dagoretti North
Mutu-ini, Ngand'o  Dagoretti South
 Riruta  Dagoretti South
 Uthiru Dagoretti South
 Nairobi West  Langata
 Nyayo Highrise  Langata
 South C Langata
Laini Saba  Kibra
 Lindi  Kibra
 Makina  Kibra
 Woodley-Kenyatta Golf Course  Kibra
 Sarang'ombe Kibra
 Kahawa West  Roysambu
 Kahawa Roysambu
Clay City  Kasarani
 Ruai Kasarani
Imara Daima  Embakasi South
 Kwa Reuben  Embakasi South
 Pipeline  Embakasi South
 Kware Embakasi South
Kariobangi North  Embakasi North
 Dandora Area I  Embakasi North
 Dandora Area II  Embakasi North
 Dandora Area III  Embakasi North
 Dandora Area IV Embakasi North
Kayole North  Embakasi Central
 Kayole NorthCentral  Embakasi Central
 Kayole South  Embakasi Central
 Komarock  Embakasi Central
 Chokaa  Embakasi Central
 Matopeni Embakasi Central
Upper Savanna  Embakasi East
 Lower Savanna

[[('Kitisuru\xa0', 'Westlands', -1.2867616, 36.7619326)],
 [(' Parklands', 'Westlands', -1.2739037, 36.81490172786343)],
 [(' Kangemi\xa0', 'Westlands', -1.2695508000000002, 36.757059931155226)],
 [(' Karura\xa0', 'Westlands', -1.2506844, 36.814611)],
 [(' Mountain View', 'Westlands', 54.9971216, -1.5919592)],
 [('Kilimani\xa0', 'Dagoretti North', -1.2874422, 36.7845225)],
 [(' Kawangware\xa0', 'Dagoretti North', -1.2992957, 36.7601909)],
 [(' Gatina\xa0', 'Dagoretti North', -1.2776591, 36.7528445)],
 [(' Kileleshwa\xa0', 'Dagoretti North', -1.2996366, 36.7594969)],
 [(' Kabiro', 'Dagoretti North', -1.2844435, 36.7462581)],
 [(' Riruta\xa0', 'Dagoretti South', -1.2968943, 36.7500698)],
 [(' Uthiru', 'Dagoretti South', -1.2601943, 36.7183773)],
 [(' Waithaka', 'Dagoretti South', -1.3138818, 36.7008996)],
 [('Karen\xa0', 'Langata', -1.3260791, 36.7095882)],
 [(' Nairobi West\xa0', 'Langata', -1.36712725, 36.751154217099554)],
 [(' Nyayo Highrise\xa0', 'Langata', -1.31600475, 36.801733448

In [6]:
nairobidata = pd.DataFrame([item for constitlist in rawdata for item in constitlist])
nairobidata.columns = ['Ward', 'Constituency', 'Latitude', 'Longitude']
nairobidata

,Ward,Constituency,Latitude,Longitude
0,Kitisuru,Westlands,-1.286762,36.761933
1,Parklands,Westlands,-1.273904,36.814902
2,Kangemi,Westlands,-1.269551,36.757060
3,Karura,Westlands,-1.250684,36.814611
4,Mountain View,Westlands,54.997122,-1.591959
5,Kilimani,Dagoretti North,-1.287442,36.784523
6,Kawangware,Dagoretti North,-1.299296,36.760191
7,Gatina,Dagoretti North,-1.277659,36.752845
8,Kileleshwa,Dagoretti North,-1.299637,36.759497
9,Kabiro,Dagoretti North,-1.284444,36.746258


### Removing duplicate datapoints and datapoints outside the latitude and longitude range for Nairobi 

In [123]:
remove = nairobidata[ (nairobidata['Latitude'] > -1.15) | (nairobidata['Latitude'] < -1.37) | (nairobidata['Longitude'] < 36.0) | (nairobidata['Longitude'] > 37.0) ].index
nairobidata.drop(remove, inplace=True)nairobidata.drop_duplicates(subset=['Latitude'],inplace=True)
nairobidata

,Ward,Constituency,Latitude,Longitude
0,Kitisuru,Westlands,-1.286762,36.761933
1,Parklands,Westlands,-1.273904,36.814902
2,Kangemi,Westlands,-1.269551,36.757060
3,Karura,Westlands,-1.250684,36.814611
5,Kilimani,Dagoretti North,-1.287442,36.784523
6,Kawangware,Dagoretti North,-1.299296,36.760191
7,Gatina,Dagoretti North,-1.277659,36.752845
8,Kileleshwa,Dagoretti North,-1.299637,36.759497
9,Kabiro,Dagoretti North,-1.284444,36.746258
10,Riruta,Dagoretti South,-1.296894,36.750070


### plotting the results on a map

In [124]:
address = 'Nairobi, Kenya'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


map_nairobi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, ward, neighborhood in zip(nairobidata['Latitude'], nairobidata['Longitude'], nairobidata['Ward'], nairobidata['Constituency']):
    label = '{}, {}'.format(ward, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nairobi)  
    
map_nairobi

In [125]:
CLIENT_ID = 'LDV43HSRTOKDMLLHT5KT0DBY2JFJ5ADMCR2YSHBJHOECAAX3' # your Foursquare ID
CLIENT_SECRET = 'HZQGB3J0H3RVHUKOJ3WBAKVI0VOSTMJ0AYR02HDFZKH1HOM5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: Hidden')
print('CLIENT_SECRET: Hidden')

Your credentails:
CLIENT_ID: Hidden
CLIENT_SECRET: Hidden


In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Finding the venues nearby to the ward locations

In [126]:
nairobi_venues = getNearbyVenues(names=nairobidata['Ward'],
                                   latitudes=nairobidata['Latitude'],
                                   longitudes=nairobidata['Longitude']
                                  )

Kitisuru 
 Parklands
 Kangemi 
 Karura 
Kilimani 
 Kawangware 
 Gatina 
 Kileleshwa 
 Kabiro
 Riruta 
 Uthiru
 Waithaka
Karen 
 Nairobi West 
 Nyayo Highrise 
 South C
Laini Saba 
 Lindi 
 Makina 
 Sarang'ombe
Githurai 
 Kahawa West 
 Roysambu 
 Kahawa
Clay City 
 Mwiki 
 Kasarani 
 Njiru Shopping 
 Ruai
Babadogo 
 Utalii 
 Mathare North 
 Lucky Summer 
 Korogocho
Imara Daima 
 Kwa Njenga 
 Kwa Reuben 
 Kware
Kariobangi North 
 Dandora Area I 
Kayole North 
 Komarock 
Upper Savanna 
 Embakasi 
 Utawala 
 Mihang'o
Umoja I 
 Umoja II 
Maringo
 Viwandani 
 Harambee 
 Makongeni
Pumwani 
 Eastleigh North 
Nairobi Central 
 Pangani 
Hospital 
 Huruma 
 Ngei 
 Mlango Kubwa 
 Kiamaiko


In [127]:
print(nairobi_venues.shape)
nairobi_venues.head()

(364, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kitisuru,-1.286762,36.761933,Toranj,-1.285340,36.762384,Persian Restaurant
1,Kitisuru,-1.286762,36.761933,Le Palanka,-1.286145,36.761896,African Restaurant
2,Kitisuru,-1.286762,36.761933,The Arbor,-1.287757,36.761574,Breakfast Spot
3,Kitisuru,-1.286762,36.761933,MumyDada - Nigerian & Kenyan Cuisine,-1.284435,36.764809,Bar
4,Kitisuru,-1.286762,36.761933,The Ambo Garden,-1.283841,36.761392,Lounge


In [128]:
nairobi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Dandora Area I,1,1,1,1,1,1
Eastleigh North,4,4,4,4,4,4
Embakasi,1,1,1,1,1,1
Harambee,10,10,10,10,10,10
Huruma,2,2,2,2,2,2
Kabiro,2,2,2,2,2,2
Kahawa,2,2,2,2,2,2
Kahawa West,4,4,4,4,4,4
Karura,4,4,4,4,4,4


In [129]:
# one hot encoding
nairobi_onehot = pd.get_dummies(nairobi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nairobi_onehot['Neighbourhood'] = nairobi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nairobi_onehot.columns[-1]] + list(nairobi_onehot.columns[:-1])
nairobi_onehot = nairobi_onehot[fixed_columns]

nairobi_onehot.head()

,Neighbourhood,African Restaurant,Antique Shop,Arcade,Art Gallery,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Garden,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Campground,Child Care Service,Chinese Restaurant,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gift Shop,Gun Range,Gym,Gym / Fitness Center,Health & Beauty Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lounge,Market,Men's Store,Metro Station,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nightclub,Nightlife Spot,North Indian Restaurant,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Persian Restaurant,Pharmacy,Pier,Pizza Place,Pool,Pub,Recreation Center,Resort,Restaurant,Rugby Pitch,Sandwich Place,Sausage Shop,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Sports Bar,Stadium,Steakhouse,Supermarket,Szechuan Restaurant,Theater,Train Station,Turkish Restaurant,Whisky Bar,Wine Bar
0,Kitisuru,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Kitisuru,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Kitisuru,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Kitisuru,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Kitisuru,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [130]:
nairobi_grouped = nairobi_onehot.groupby('Neighbourhood').mean().reset_index()
nairobi_grouped.head()

,Neighbourhood,African Restaurant,Antique Shop,Arcade,Art Gallery,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Garden,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Campground,Child Care Service,Chinese Restaurant,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gift Shop,Gun Range,Gym,Gym / Fitness Center,Health & Beauty Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lounge,Market,Men's Store,Metro Station,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nightclub,Nightlife Spot,North Indian Restaurant,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Persian Restaurant,Pharmacy,Pier,Pizza Place,Pool,Pub,Recreation Center,Resort,Restaurant,Rugby Pitch,Sandwich Place,Sausage Shop,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Sports Bar,Stadium,Steakhouse,Supermarket,Szechuan Restaurant,Theater,Train Station,Turkish Restaurant,Whisky Bar,Wine Bar
0,Dandora Area I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Eastleigh North,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Embakasi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Harambee,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.00,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Huruma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### creating a table with the most common venues for each location

In [175]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = nairobi_grouped['Neighbourhood']

for ind in np.arange(nairobi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nairobi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Dandora Area I,Campground,Wine Bar,Food Court,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant,Fast Food Restaurant
1,Eastleigh North,African Restaurant,Bus Station,Flea Market,Food Court,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant
2,Embakasi,Lounge,Wine Bar,French Restaurant,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant,Fast Food Restaurant
3,Harambee,African Restaurant,Steakhouse,Ethiopian Restaurant,Lounge,Shopping Mall,Fried Chicken Joint,Nightclub,Pizza Place,Pharmacy,Convenience Store
4,Huruma,Restaurant,Bar,Wine Bar,Food Court,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant


In [176]:
# set number of clusters
kclusters = 6

nairobi_grouped_clustering = nairobi_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nairobi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [177]:
neighborhoods_venues_sorted = neighborhoods_venues_sorted.rename(columns={'Neighbourhood':'Ward'})

nairobi_merged = nairobidata

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
nairobi_merged = nairobi_merged.join(neighborhoods_venues_sorted.set_index('Ward'), on='Ward')

nairobi_merged = nairobi_merged.dropna()
nairobi_merged = nairobi_merged.astype({"Cluster Labels": int})
 # check the last columns!
nairobi_merged

,Ward,Constituency,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Kitisuru,Westlands,-1.286762,36.761933,3,African Restaurant,Bar,Korean Restaurant,Lounge,Diner,Chinese Restaurant,Breakfast Spot,Beer Garden,Persian Restaurant,Bakery
1,Parklands,Westlands,-1.273904,36.814902,3,Hotel,Department Store,Bookstore,Gastropub,Hotel Bar,Indian Restaurant,Museum,Nightclub,Coffee Shop,Pub
3,Karura,Westlands,-1.250684,36.814611,0,Hotel,Boutique,North Indian Restaurant,Park,Gun Range,Gift Shop,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store
5,Kilimani,Dagoretti North,-1.287442,36.784523,0,Hotel,Lounge,Seafood Restaurant,Gym,Gaming Cafe,Japanese Restaurant,Ethiopian Restaurant,Music Venue,Nightclub,Gym / Fitness Center
6,Kawangware,Dagoretti North,-1.299296,36.760191,3,Bar,Fast Food Restaurant,Wine Bar,Shopping Mall,Coffee Shop,Gift Shop,Lounge,Italian Restaurant,Ice Cream Shop,Gym / Fitness Center
8,Kileleshwa,Dagoretti North,-1.299637,36.759497,3,Bar,Fast Food Restaurant,Wine Bar,Shopping Mall,Coffee Shop,Gift Shop,Lounge,Italian Restaurant,Ice Cream Shop,Gym / Fitness Center
9,Kabiro,Dagoretti North,-1.284444,36.746258,3,Cocktail Bar,Movie Theater,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant,Fast Food Restaurant,Fish & Chips Shop
10,Riruta,Dagoretti South,-1.296894,36.750070,3,Italian Restaurant,Auto Garage,Wine Bar,French Restaurant,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant
11,Uthiru,Dagoretti South,-1.260194,36.718377,0,Gym,Pub,Dry Cleaner,Bus Station,Wine Bar,Food & Drink Shop,Cosmetics Shop,Cricket Ground,Department Store,Diner
12,Waithaka,Dagoretti South,-1.313882,36.700900,1,Bar,Antique Shop,Other Great Outdoors,Italian Restaurant,Wine Bar,Food Court,Cricket Ground,Department Store,Diner,Dry Cleaner


In [178]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nairobi_merged['Latitude'], nairobi_merged['Longitude'], nairobi_merged['Ward'], nairobi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [184]:
cluster0 = nairobi_merged[nairobi_merged['Cluster Labels']==0]
cluster1 = nairobi_merged[nairobi_merged['Cluster Labels']==1]
cluster2 = nairobi_merged[nairobi_merged['Cluster Labels']==2]
cluster3 = nairobi_merged[nairobi_merged['Cluster Labels']==3]
cluster4 = nairobi_merged[nairobi_merged['Cluster Labels']==4]
cluster5 = nairobi_merged[nairobi_merged['Cluster Labels']==5]
cluster6 = nairobi_merged[nairobi_merged['Cluster Labels']==6]

In [185]:
cluster0

,Ward,Constituency,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Karura,Westlands,-1.250684,36.814611,0,Hotel,Boutique,North Indian Restaurant,Park,Gun Range,Gift Shop,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store
5,Kilimani,Dagoretti North,-1.287442,36.784523,0,Hotel,Lounge,Seafood Restaurant,Gym,Gaming Cafe,Japanese Restaurant,Ethiopian Restaurant,Music Venue,Nightclub,Gym / Fitness Center
11,Uthiru,Dagoretti South,-1.260194,36.718377,0,Gym,Pub,Dry Cleaner,Bus Station,Wine Bar,Food & Drink Shop,Cosmetics Shop,Cricket Ground,Department Store,Diner
34,Lucky Summer,Ruaraka,-1.239308,36.898786,0,Gym,Soccer Field,Market,Wine Bar,Food & Drink Shop,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store,Diner
36,Imara Daima,Embakasi South,-1.328590,36.874400,0,Train Station,Gym,Café,Wine Bar,Food Court,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner
47,Komarock,Embakasi Central,-1.268377,36.904706,0,Gym / Fitness Center,Gym,Pizza Place,Shopping Mall,Wine Bar,Flea Market,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store
65,Nairobi Central,Starehe,-1.279296,36.840374,0,African Restaurant,Cosmetics Shop,Soccer Field,Lounge,Moving Target,Food Court,Cricket Ground,Department Store,Diner,Dry Cleaner


In [186]:
cluster1

,Ward,Constituency,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Waithaka,Dagoretti South,-1.313882,36.700900,1,Bar,Antique Shop,Other Great Outdoors,Italian Restaurant,Wine Bar,Food Court,Cricket Ground,Department Store,Diner,Dry Cleaner
28,Kasarani,Kasarani,-1.223271,36.905904,1,Bar,Wine Bar,French Restaurant,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant,Fast Food Restaurant
30,Ruai,Kasarani,-1.268997,36.991548,1,Bar,Restaurant,Shopping Mall,Wine Bar,Food & Drink Shop,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store,Diner
33,Mathare North,Ruaraka,-1.251788,36.870772,1,Bar,Fish & Chips Shop,Wine Bar,French Restaurant,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant
72,Huruma,Mathare,-1.258309,36.876976,1,Restaurant,Bar,Wine Bar,Food Court,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant


In [187]:
cluster2

,Ward,Constituency,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Embakasi,Embakasi East,-1.318697,36.884473,2,Lounge,Wine Bar,French Restaurant,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant,Fast Food Restaurant


In [188]:
cluster3

,Ward,Constituency,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Kitisuru,Westlands,-1.286762,36.761933,3,African Restaurant,Bar,Korean Restaurant,Lounge,Diner,Chinese Restaurant,Breakfast Spot,Beer Garden,Persian Restaurant,Bakery
1,Parklands,Westlands,-1.273904,36.814902,3,Hotel,Department Store,Bookstore,Gastropub,Hotel Bar,Indian Restaurant,Museum,Nightclub,Coffee Shop,Pub
6,Kawangware,Dagoretti North,-1.299296,36.760191,3,Bar,Fast Food Restaurant,Wine Bar,Shopping Mall,Coffee Shop,Gift Shop,Lounge,Italian Restaurant,Ice Cream Shop,Gym / Fitness Center
8,Kileleshwa,Dagoretti North,-1.299637,36.759497,3,Bar,Fast Food Restaurant,Wine Bar,Shopping Mall,Coffee Shop,Gift Shop,Lounge,Italian Restaurant,Ice Cream Shop,Gym / Fitness Center
9,Kabiro,Dagoretti North,-1.284444,36.746258,3,Cocktail Bar,Movie Theater,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant,Fast Food Restaurant,Fish & Chips Shop
10,Riruta,Dagoretti South,-1.296894,36.750070,3,Italian Restaurant,Auto Garage,Wine Bar,French Restaurant,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant
13,Karen,Langata,-1.326079,36.709588,3,Shopping Mall,Café,Coffee Shop,Gym / Fitness Center,Indian Restaurant,Pub,Japanese Restaurant,Snack Place,Italian Restaurant,Supermarket
14,Nairobi West,Langata,-1.367127,36.751154,3,Art Gallery,Café,Garden,Resort,Wine Bar,Food & Drink Shop,Cosmetics Shop,Cricket Ground,Department Store,Diner
15,Nyayo Highrise,Langata,-1.316005,36.801733,3,Lounge,Athletics & Sports,Gun Range,BBQ Joint,Bar,Wine Bar,Fast Food Restaurant,Food & Drink Shop,Flea Market,Fish & Chips Shop
16,South C,Langata,-1.320401,36.830525,3,Coffee Shop,Beer Garden,Wine Bar,Food Court,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant


In [189]:
cluster4

,Ward,Constituency,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Kwa Reuben,Embakasi South,-1.314298,36.869721,4,Recreation Center,Wine Bar,Cocktail Bar,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant
41,Kariobangi North,Embakasi North,-1.315138,36.867340,4,Recreation Center,Wine Bar,Cocktail Bar,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant


In [190]:
cluster5

,Ward,Constituency,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Makina,Kibra,-1.311237,36.786862,5,Athletics & Sports,Wine Bar,French Restaurant,Cosmetics Shop,Cricket Ground,Department Store,Diner,Dry Cleaner,Ethiopian Restaurant,Fast Food Restaurant
